# Annotation der Volksverhetzung im Referenzdatensatz

1. Ausschnittdatensätze (Train, Test) erstellen, in denen nur die Einträge mit dem feinen Label "HATE" vorkommen

In [ ]:
with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train_OD.txt", mode="r", encoding="utf-8") as hatetrainin:
    all_cont = hatetrainin.readlines()
    all_cont = all_cont[1:]
    sep_cont = [entry.strip().split("\t") for entry in all_cont]
    hate = []
    for tweet in sep_cont:
        if tweet[3] == "HATE": hate.append(tweet)

# Als Textdatei
with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train_HATE.txt", mode="w", encoding="utf-8") as hatetrainout:
    for line in hate:
        hatetrainout.write("\t".join(line)+"\n")

# Als json-Datei
import json
cont_dicts = [{"id":elem[0], "text":elem[1], "tag1":elem[2], "tag2":elem[3]} for elem in hate]

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train_HATE.json", mode="w", encoding="utf-8") as jsonout:
    prep_cont = json.dumps(cont_dicts)
    jsonout.write(prep_cont)

2. Annotation in doccano

Ergebnisse

Anmerkungen:
    - in diesen json-Dateien sind noch Tweets enthalten, die später als Duplikate überall sonst entfernt wurden;
        diese werden in der weiteren Verarbeitung daher ignoriert
    - anfangs war die Überlegung den Split der Train- und Testdaten so zu belassen wie durch die Datensätze vorgegeben;
        um stratifizierte Verhältnisse in Split zu ermöglichen, wird ab hier aber mit dem gesamten Datensatz HATE gearbeitet;
        der Train/Test-Split erfolgt später

3. Prüfung der Annotationshierarchie auf Korrektheit

In [ ]:
def check_anno(datensatz):
    """Annotationslogik checken und Problemfälle sammeln"""
    probleme = []
    for entry in datensatz:
        if not anno_logik_check(entry["label"]): probleme.append(entry)
    return probleme


def anno_logik_check(labelset):
    """ Die Annotationslogik checken:
        Input: Labelmenge als Liste
        Output: True (falls alles korrekt) / False (falls irgendein Problem vorliegt)    
    """
    labelset = set(labelset)
    korrekt = True
    if len(labelset) == 0:
        korrekt = False
    elif (len(labelset) == 1) and ("KEINE" not in labelset):
        korrekt = False
    else:
        # 1. VVH-ALLG interne Logik
        if "VVH-ALLG" in labelset:
            # Mind. eine Art der Tathandlung ggb.
            if {"Aufstachelung zu Hass", "Aufforderung zu Gewalt- oder Willkürmaßnahmen", "Angriff der Menschenwürde"} & labelset == set():
                korrekt = False
            # Mind. eine Gruppe genannt
            if {"Nationalität", 'ethnische Herkunft / "Rasse"', "Religion / Weltanschauung",
                    "Politische Einstellung", "Geschlecht", "Anderes Merkmal"} & labelset == set():
                korrekt = False
            # keine VVH-NS Labels
            if {"VVH-NS", "Billigen", "Verherrlichen", "Verharmlosen", "Leugnen", "Rechtfertigen"} & labelset != set():
                korrekt = False
            if "KEINE" in labelset: korrekt = False

        # 2. VVH-NS interne Logik
        if "VVH-NS" in labelset:
            # Mind. eine Art der Tathandlung ggb.
            if {"Billigen", "Verherrlichen", "Verharmlosen", "Leugnen", "Rechtfertigen"} & labelset == set(): 
                korrekt = False
            # keine VVH-ALLG Labels
            if {"VVH-ALLG", "Aufstachelung zu Hass", "Aufforderung zu Gewalt- oder Willkürmaßnahmen", "Angriff der Menschenwürde"} & labelset != set():
                korrekt = False 
            if "KEINE" in labelset: korrekt = False
        
        # 3. Gruppenmerkmale - Logik (in beide Richtungen)

        nation = {"Türkischstämmige Deutsche", "Marokkaner:innen", "In Deutschland lebende Ausländer:innen",
                    "Asiat:innen", "Pol:innen", "Afrikaner:innen", "Syrer:innen", "Palästinenser:innen"}
        herkunft = {"POC", "Araber:in"}
        religion = {"Muslim:e/innen", "Juden/Jüdinnen", "Christ:innen"}
        polit = {"Die Grünen", "die SPD", "die Linke", "CDU/CSU", "AfD", "Nazis", "Islamist:innen", "Kommunist:innen",
                    "Zionist:innen", "NPD", "FDP", "FridaysForFuture", "Pegida", "Anarchist:in"}
        geschlecht = {"Trans/NB-Personen", "Frauen", "Männer"}
        andere = {"Flüchtlinge", "Asylbewerber:innen", "Sich illegal in Deutschland aufhaltende Personen", "Migrant:innen", "Vorbestrafte",
                    "Veganer", "Senior:innen", "Lesben, Schwule, Bi", "Kinder", "Jugendliche", "Polizist:innen", "Obdachlose",
                    "Richter:innen", "Analphabet:innen", "Soldat:innen", "Behinderte"}

        # Richtung 1: falls Gruppe vorhanden, korrektes Merkmal auch vorhanden
        for i in nation & labelset:
            if "Nationalität" not in labelset:
                korrekt = False
        for j in herkunft & labelset:
            if 'ethnische Herkunft / "Rasse"' not in labelset:
                korrekt = False
        for k in religion & labelset:
            if "Religion / Weltanschauung" not in labelset:
                korrekt = False
        for l in polit & labelset:
            if "Politische Einstellung" not in labelset:
                korrekt = False
        for m in geschlecht & labelset:
            if "Geschlecht" not in labelset:
                korrekt = False
        for n in andere & labelset:
            if "Anderes Merkmal" not in labelset:
                korrekt = False

        # Richtung 2: falls Merkmal vorhanden, auch eine passende Gruppe vorhanden
        if "Nationalität" in labelset:
            if nation & labelset == set():
                korrekt = False
        if 'ethnische Herkunft / "Rasse"' in labelset:
            if herkunft & labelset == set():
                korrekt = False
        if "Religion / Weltanschauung" in labelset:
            if religion & labelset == set():
                korrekt = False
        if "Politische Einstellung" in labelset:
            if polit & labelset == set():
                korrekt = False
        if "Geschlecht" in labelset:
            if geschlecht & labelset == set():
                korrekt = False
        if "Anderes Merkmal" in labelset:
            if andere & labelset == set():
                korrekt = False

    return korrekt

In [ ]:
# Probleme in der Annotationslogik händisch durchgehen

# Jeweils für die Trainingsdaten und die Testdaten
# Beispieleintrag der Annotations-Json-Datei:
# {"id": "01112520", "data": "@SteiblBarbara @Thomas_S_Wagner @RitaKratzert Weitaus schlimmer. Heute ist es nicht mehr Dummheit.  Ein ganzes Volk ist zu (m)wutlosen Zombies dressiert worden.", "label": ["KEINE"], "tag1": "NEG", "tag2": "HATE"}
with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train_HATE_annotiert.json", mode="r", encoding="utf-8") as trainf:
    train_anno = trainf.read()
    train_annotations = json.loads(train_anno)

probleme = check_anno(train_annotations)

In [ ]:
# Korrekturen: jeder fehlerhafte Eintrag wurde einzeln von Hand korrigiert

new01110262 = {'id': '01110262', 'label': ['KEINE', 'Flüchtlinge', 'Afrikaner:innen', 'Nationalität', 'Anderes Merkmal']}
new03113327 = {'id': '03113327', 'label': ['Angriff der Menschenwürde', 'VVH-ALLG', 'POC', 'ethnische Herkunft / "Rasse"', 'Kinder', 'Die Grünen', 'Politische Einstellung', 'Anderes Merkmal'] }
new01114326 = {'id': '01114326', 'label': ['Muslim:e/innen', 'Religion / Weltanschauung', 'Islamist:innen', 'Aufstachelung zu Hass', 'VVH-ALLG', 'Politische Einstellung']}
new01111028 = {'id': '01111028', 'label': ['KEINE', 'Araber:in', 'ethnische Herkunft / "Rasse"']}
new03113648 = {'id': '03113648', 'label': ['KEINE', 'Politische Einstellung', 'Nazis']}
new01112254 = {'id': '01112254', 'label': ['KEINE', 'Pol:innen', 'In Deutschland lebende Ausländer:innen', 'Nationalität']}
new02110710 = {'id': '02110710', 'label': ['KEINE', 'Araber:in', 'In Deutschland lebende Ausländer:innen', 'ethnische Herkunft / "Rasse"', 'POC', 'Afrikaner:innen', 'Nationalität']}
new04111621 = {'id': '04111621', 'label': ['KEINE']}
new03112728 = {'id': '03112728', 'label': ['KEINE']}

# Entspricht damit nicht mehr der ursprünglichen Logik; Entscheidung: Palästinenser:innen eher Nationalität als Polit. Einstellung
# >> alle entsprechenden Einträge umschreiben
new02113786 = {'id': '02113786', 'label': ['KEINE', 'Palästinenser:innen', 'Nationalität']}
new01113048 = {'id': '01113048', 'label': ['Palästinenser:innen', 'Nationalität', 'VVH-ALLG', 'Aufstachelung zu Hass'], 'tag1': 'NEG', 'tag2': 'HATE'}
new01110156 = {'id': '01110156', 'label': ['Palästinenser:innen', 'Nationalität', 'VVH-ALLG', 'Aufstachelung zu Hass'], 'tag1': 'NEG', 'tag2': 'HATE'}

new01222281 = {'id': '01222281', 'label': ['Die Grünen', 'Türkischstämmige Deutsche','Nationalität', 'Politische Einstellung', 'KEINE']}
new02220290 = {'id': '02220290', 'label': ['KEINE', 'die SPD', 'Politische Einstellung']}
new01221146  = {'id': '01221146', 'label': ['KEINE', 'die SPD', 'Politische Einstellung']}
new01220517 = {'id': '01220517', 'label': ['KEINE', 'In Deutschland lebende Ausländer:innen', 'Pol:innen', 'Migrant:innen', 'Nationalität', 'Anderes Merkmal']}
new02221882 = {'id': '02221882', 'label': ['KEINE', 'Zionist:innen', 'Palästinenser:innen', 'Politische Einstellung', 'Nationalität']}
new01223120 = {'id': '01223120', 'label': ['KEINE']}
new01221025 = {'id': '01221025', 'label': ['Verharmlosen', 'VVH-NS']}
new02222036 = {'id': '02222036', 'label': ['KEINE', 'Palästinenser:innen', 'Nationalität']}
new01220250 = {'id': '01220250', 'label': ['KEINE', 'Palästinenser:innen', 'Nationalität']}
new02221762 = {'id': '02221762', 'label': ['KEINE', 'AfD', 'Politische Einstellung', 'Islamist:innen', 'Araber:in', 'In Deutschland lebende Ausländer:innen', 'Nationalität', 'ethnische Herkunft / "Rasse"']}
new01223162 = {'id': '01223162', 'label': ['KEINE', 'Islamist:innen', 'Politische Einstellung']}
new02221146 = {'id': '02221146', 'label': ['Nazis', 'Aufforderung zu Gewalt- oder Willkürmaßnahmen', 'VVH-ALLG', 'Politische Einstellung']}
new01220249 = {'id': '01220249', 'label': ['KEINE', 'In Deutschland lebende Ausländer:innen', 'Nationalität']}
new02221991 = {'id': '02221991', 'label': ['KEINE', 'Palästinenser:innen', 'Zionist:innen', 'Politische Einstellung', 'Nationalität']}
new02220270 = {'id': '02220270', 'label': ['KEINE', 'Araber:in', 'Afrikaner:innen', 'In Deutschland lebende Ausländer:innen', 'Nationalität', 'Religion / Weltanschauung', 'Muslim:e/innen', 'ethnische Herkunft / "Rasse"']}
new02220311 = {'id': '02220311', 'label': ['KEINE', 'Araber:in', 'In Deutschland lebende Ausländer:innen', 'Afrikaner:innen', 'Muslim:e/innen', 'Religion / Weltanschauung', 'Nationalität', 'ethnische Herkunft / "Rasse"']}
new01222167 = {'id': '01222167', 'label': ['KEINE', 'In Deutschland lebende Ausländer:innen', 'Islamist:innen', 'Die Grünen', 'Politische Einstellung', 'Asylbewerber:innen', 'Anderes Merkmal', 'Nationalität']}
new01222700 = {'id': '01222700', 'label': ['KEINE', 'Anderes Merkmal', 'Sich illegal in Deutschland aufhaltende Personen']}
new01222612 = {'id': '01222612', 'label': ['KEINE', 'Palästinenser:innen', 'Nationalität']}

In [ ]:
# Erkannte Fehler: Tweets korrigiert in neue Datei schreiben, inkl. der Korrektur des Off-by-One-Fehlers in den IDs (je +1)

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train_HATE_annotiert.json", mode="r", encoding="utf-8") as trainf:
    train_anno = trainf.read()
    train_annotations = json.loads(train_anno)

korrekturen_train = [new01110262, new03113327, new01114326, new01111028, new03113648, new01112254, new02110710, new04111621, new03112728, new02113786, new01113048, new01110156]
for i in train_annotations:
    for j in korrekturen_train:
        if i["id"] == j["id"]:
            i["label"] = j["label"]
    i["id"] = i["id"][:4] + f'{int(i["id"][4:])+1:04d}'


train_anno_k = json.dumps(train_annotations)

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train_HATE_anno_korrigiert.json", mode="w", encoding="utf-8") as trainf_k:
    trainf_k.write(train_anno_k)


with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test_HATE_annotiert.json", mode="r", encoding="utf-8") as testf:
    test_anno = testf.read()
    test_annotations = json.loads(test_anno)

korrekturen_test = [new01222281, new02220290, new01221146, new01220517, new02221882, new01223120, new01221025, new02222036, new01220250,
                    new02221762, new01223162, new02221146, new01220249, new02221991, new02220270, new02220311, new01222167, new01222700, new01222612]

for i in test_annotations:
    for j in korrekturen_test:
        if i["id"] == j["id"]:
            i["label"] = j["label"]
    i["id"] = i["id"][:4] + f'{int(i["id"][4:])+1:04d}'

test_anno_k = json.dumps(test_annotations)

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test_HATE_anno_korrigiert.json", mode="w", encoding="utf-8") as testf_k:
    testf_k.write(test_anno_k)

3. Annotationen formatieren und speichern

- in unterschiedlicher Detailgenauigkeit speichern:
    - VVH Ja/Nein, VVH-Allg Ja/Nein, VVH-NS Ja/Nein
    - Gruppe Ja/Nein, Welches Gruppenmerkmal/Keine Gruppe, Welche Gruppe genau/Keine Gruppe 
    - Handlung VVH-Allg Ja/Nein, Welche Handlung genau/Keine Handlung
    - Handlung VVH-NS Ja/Nein, Welche Handlung genau/Keine Handlung

- gleichzeitig @user-Erwähnungen anonymisieren


In [1]:
# Liste der Duplikate laden (schwarze Liste)
with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\schwarze_Liste.txt", mode="r", encoding="utf-8") as schw:
    schwarzeListe = schw.readlines()
    schwarzeListe = [line.strip() for line in schwarzeListe]

import re

def anonym_atuser(tweet):
    tweet_anonym = re.sub('@[^@ ]+?@', '@user@', tweet)
    tweet_anonym = re.sub('@[^@ ]+? ', '@user ', tweet_anonym)
    tweet_anonym = re.sub('@[^@ ]+?$', '@user', tweet_anonym)
    return tweet_anonym
    

In [ ]:
# TODO: Train und Test jeweils gesammelt in eine Datei schreiben; KEINE Trennung (nur eben implizit über die IDs)

In [3]:
import json
import copy

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train_HATE_anno_korrigiert.json", mode="r", encoding="utf-8") as in_hate_train:
    hate_train = in_hate_train.read()
    hate_train_anno = json.loads(hate_train)
    for entry in hate_train_anno:
        entry["data"] = anonym_atuser(entry["data"])


with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test_HATE_anno_korrigiert.json", mode="r", encoding="utf-8") as in_hate_test:
    hate_test = in_hate_test.read()
    hate_test_anno = json.loads(hate_test)
    for entry in hate_test_anno:
        entry["data"] = anonym_atuser(entry["data"])

hate_anno = hate_train_anno + hate_test_anno

# Beispiel-Eintrag: {"id": "01222380", "data": "den #Zentralrat der #Muslime sollte man komplett rausschmei\u00dfen, er hat hier nichts zu suchen @aktuelleStunde #WDR", "label": ["KEINE"], "tag1": "NEG", "tag2": "HATE"}

# Verallgemeinerte Annotation

# 1. VVH Ja/Nein:
def transform_anno_vvh(corp):
    new_anno = []
    for entry in corp:
        new_entry = copy.deepcopy(entry)
        if "KEINE" in entry["label"]:
            new_entry["label"] = "KEINE"
        else:
            new_entry["label"] = "VVH"
        new_anno.append(new_entry)
    return new_anno

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_HATE_VVH.txt", mode="w", encoding="utf-8") as out_vvh:
    for i in transform_anno_vvh(hate_anno):
        if i["id"] not in schwarzeListe:
            out_vvh.write("\t".join((i["id"], i["data"], i["label"]))+"\n")


# 2.1   Gruppe Ja/Nein
def transform_anno_gruppe(corp):
    gruppe = {"Nationalität", 'ethnische Herkunft / "Rasse"', "Religion / Weltanschauung",
                    "Politische Einstellung", "Geschlecht", "Anderes Merkmal"}
    new_anno = []
    for entry in corp:
        new_entry = copy.deepcopy(entry)
        if gruppe & set(entry["label"]) != set():
            new_entry["label"] = "Gruppe"
        else:
            new_entry["label"] = "KeineGruppe"
        new_anno.append(new_entry)
    return new_anno

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_HATE_Gruppe.txt", mode="w", encoding="utf-8") as out_gr:
    for i in transform_anno_gruppe(hate_anno):
        if i["id"] not in schwarzeListe:
            out_gr.write("\t".join((i["id"], i["data"], i["label"]))+"\n")


# 2.2   Welche Gruppe / Keine Gruppe
def transform_anno_gruppe_det(corp):
    gruppe = {"Nationalität", 'ethnische Herkunft / "Rasse"', "Religion / Weltanschauung",
                    "Politische Einstellung", "Geschlecht", "Anderes Merkmal"}
    new_anno = []
    for entry in corp:
        new_entry = copy.deepcopy(entry)
        if gruppe & set(entry["label"]) != set():
            new_entry["label"] = list(gruppe & set(entry["label"]))
        else:
            new_entry["label"] = ["KeineGruppe"]
        new_anno.append(new_entry)
    return new_anno

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_HATE_GruppeDetail.json", mode="w", encoding="utf-8") as out_grd:
    gruppedetail = [entry for entry in transform_anno_gruppe_det(hate_anno) if entry["id"] not in schwarzeListe]
    grdt = json.dumps(gruppedetail)
    out_grd.write(grdt)


# 3.1   Handlung VVH-Allg Ja/Nein
def transform_anno_handlung(corp):
    handlung = {"Aufstachelung zu Hass", "Aufforderung zu Gewalt- oder Willkürmaßnahmen", "Angriff der Menschenwürde"}
    new_anno = []
    for entry in corp:
        new_entry = copy.deepcopy(entry)
        if handlung & set(entry["label"]) != set():
            new_entry["label"] = "Handlung"
        else:
            new_entry["label"] = "KeineHandlung"
        new_anno.append(new_entry)
    return new_anno

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_HATE_Handlung.txt", mode="w", encoding="utf-8") as out_handl:
    for i in transform_anno_handlung(hate_anno):
        if i["id"] not in schwarzeListe:
            out_handl.write("\t".join((i["id"], i["data"], i["label"]))+"\n")


# 3.2   Welche Handlung VVH-Allg / Keine Handlung 
def transform_anno_handlung_det(corp):
    handlung = {"Aufstachelung zu Hass", "Aufforderung zu Gewalt- oder Willkürmaßnahmen", "Angriff der Menschenwürde"}
    new_anno = []
    for entry in corp:
        new_entry = copy.deepcopy(entry)
        if handlung & set(entry["label"]) != set():
            new_entry["label"] = list(handlung & set(entry["label"]))
        else:
            new_entry["label"] = ["KeineHandlung"]
        new_anno.append(new_entry)
    return new_anno

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Train_HATE_HandlungDetail.json", mode="w", encoding="utf-8") as out_handld:
    handdetail = [entry for entry in transform_anno_handlung_det(hate_anno) if entry["id"] not in schwarzeListe]
    hddt = json.dumps(handdetail)
    out_handld.write(hddt)

['Politische Einstellung']
['Politische Einstellung']
['Politische Einstellung']
['Nationalität']
['Nationalität']
['Anderes Merkmal']
['Anderes Merkmal']
['Politische Einstellung']
['Politische Einstellung']
['Geschlecht']
['Anderes Merkmal']
['Religion / Weltanschauung']
['Anderes Merkmal']
['Nationalität']
['Anderes Merkmal']
['Nationalität']
['Politische Einstellung']
['Religion / Weltanschauung']
['Politische Einstellung']
['Anderes Merkmal']
['Politische Einstellung']
['Nationalität']
['Religion / Weltanschauung']
['Nationalität']
['Anderes Merkmal']
['Politische Einstellung']
['Politische Einstellung']
['Anderes Merkmal']
['Politische Einstellung']
['Politische Einstellung']
['Anderes Merkmal']
['Nationalität']
['Nationalität']
['Nationalität']
['Politische Einstellung']
['Nationalität']
['Politische Einstellung']
['Anderes Merkmal']
['Religion / Weltanschauung', 'ethnische Herkunft / "Rasse"']
['Politische Einstellung']
['Politische Einstellung']
['Religion / Weltanschauung']
[

In [8]:
txtGruppe = ["..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Train_HATE_Gruppe.txt",
            "..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Test_HATE_Gruppe.txt",
            ]

txtHandlung = ["..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Train_HATE_Handlung.txt",
            "..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Test_HATE_Handlung.txt"
            ]

txtPfade = ["..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_HATE_VVH.txt",
            "..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_HATE_Gruppe.txt",
            "..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_HATE_Handlung.txt",
            ]

import json

# TODO: json-Format angleichen an das json-Format des großen Datensatzes
#     reftrainout.write("corpus_id\ttweet\tbinarylabel\tfinelabel\n")
for datei in txtPfade:
    with open(datei, mode="r", encoding="utf8") as in_txt:
        inhalt = in_txt.readlines()
        sep_inhalt = [eintrag.strip().split("\t") for eintrag in inhalt]
        json_inhalt = json.dumps(sep_inhalt)
        with open(datei.strip("txt")+"json", mode="w", encoding="utf-8") as out_json:
            out_json.write(json_inhalt)

In [ ]:
# Formatierung als BERT-fähige json-Datei (falls nötig)